In [1]:
import pandas as pd
import yaml # pip install PyYAML
import json
import sqlite3

# Data formats
In this exercise we will explore various formats for storing data of steel properties, including nominal chemical composition and some specified mechanical properties. We will explore the following formats:
* Excel spreadsheet
* Pandas DataFrame
* JSON
* YAML
* XML
* SQL (extra) 

At the same time, we will prepare the data for our next lecture on Machine Learning.

## Initial import (from Excel)
The initial dataset is stored in the Excel spreadsheet format (.xlsx), a common situation in industrial practice since Microsoft Excel is installed on effectively any device and its functionality is well-known to nearly everybody. Excel spreadsheets have the advantage of being very flexible in the way data is stored (column- or row-wise, merged cells, differently formatted cells, etc.). This, however, comes with the disadvantage that the structure of the data is not inherently clear to others. 

For data handling in Python, Pandas DataFrames are the de-facto standard today. They offer similar flexibility as Excel spreadsheets while offering ways to declare the data sructure more clearly at the same time. One important convention is to store same kind of information (i.e., properties, features) in the same column. The datatype (int, float, str) is then identical for all values in a column. All information belonging to the same measurement/sample are stored in the same row. The "column header" describes the kind of information within the column and the "index" describes the sample/measurement. Finer structure of the rows and columns is possible by using `MultiIndex`.

We begin our example by loading the sample dataset:

In [46]:
df = pd.read_excel('steel-data.xlsx')
df.head()

,Type of Steel,EN,ASTM,UNS,JIS,c(Fe) (%),c(C) (%),c(N) (%),c(Cr) (%),c(Ni) (%),...,Others,Product,Rp0.2 (MPa),Rp1.0 (MPa),Rm (MPa),A (%),min(Rp0.2@100) (MPa),min(Rp0.2@200) (MPa),min(Rp0.2@300) (MPa),min(Rp0.2@400) (MPa)
0,Ferritic Steel,1.4600,NaN,NaN,NaN,87.930,0.020,NaN,11.2,0.85,...,Ti,H,435.0,460.0,545.0,25.0,375.0,NaN,NaN,NaN
1,Ferritic Steel,1.4512,409,NaN,SUS409,88.280,0.020,NaN,11.5,0.20,...,Ti,C,255.0,275.0,425.0,33.0,220.0,200.0,190.0,NaN
2,Ferritic Steel,1.4003,NaN,S40977,NaN,87.980,0.020,NaN,11.5,0.50,...,NaN,C,355.0,375.0,525.0,25.0,320.0,240.0,230.0,NaN
3,Ferritic Steel,1.4000,410S,S41008,SUS 403,87.470,0.030,NaN,12.5,NaN,...,NaN,P,270.0,320.0,490.0,30.0,230.0,220.0,210.0,190.0
4,Ferritic Steel,1.4589,NaN,S42035,NaN,84.055,0.045,NaN,14.0,1.65,...,Ti,C,470.0,510.0,600.0,20.0,420.0,NaN,NaN,NaN


The list of column headers can be accessed, as follows:

In [47]:
df.columns

Index(['Type of Steel', 'EN', 'ASTM', 'UNS', 'JIS', 'c(Fe) (%)', 'c(C) (%)',
       'c(N) (%)', 'c(Cr) (%)', 'c(Ni) (%)', 'c(Mo) (%)', 'c(Al) (%)',
       'c(Mn) (%)', 'c(Cu) (%)', 'c(Si) (%)', 'Others', 'Product',
       'Rp0.2 (MPa)', 'Rp1.0 (MPa)', 'Rm (MPa)', 'A (%)',
       'min(Rp0.2@100) (MPa)', 'min(Rp0.2@200) (MPa)', 'min(Rp0.2@300) (MPa)',
       'min(Rp0.2@400) (MPa)'],
      dtype='object')

We now prepare the data for Machine Learning (ML) in such a way that we fill missing information on chemical concentrations with zeros instead of `NaN`s (`NaN`s cannot be treated by the ML algorithms and the whole data row would be useless):

In [48]:
cols_conc = [c for c in df.columns if 'c(' in c[:2]]
cols_prop = [c for c in df.columns if 'R' in c[:1] or 'A ' in c[:2]]
df[cols_conc] = df[cols_conc].fillna(0.0)

## Storing as JSON file
JSON ("JavaScript Object Notation") is a compact data fromat in an easy-to-read text notation for data exchange purposes (machine-machine communication). The fundamental structure of a JSON document is the one of nested Python dictionaries and lists.

Pandas DataFrames can be natively exported to JSON files:

In [50]:
df.to_json('steel-data.json', orient="records", indent=2)

Above settings for the parameters `orient` and `indent` increase the readibility of the resulting file, which looks as follows:

In [51]:
with open('steel-data.json', 'r') as f:
    content = f.read()
print(content[:234], '\n\n[...]\n', content[-230:])

[
  {
    "Type of Steel":"Ferritic Steel",
    "EN":1.46,
    "ASTM":null,
    "UNS":null,
    "JIS":null,
    "c(Fe) (%)":87.93,
    "c(C) (%)":0.02,
    "c(N) (%)":0.0,
    "c(Cr) (%)":11.2,
    "c(Ni) (%)":0.85,
    "c(Mo) (%)":0. 

[...]
 
    "Rp0.2 (MPa)":460.0,
    "Rp1.0 (MPa)":490.0,
    "Rm (MPa)":860.0,
    "A (%)":60.0,
    "min(Rp0.2@100) (MPa)":430.0,
    "min(Rp0.2@200) (MPa)":350.0,
    "min(Rp0.2@300) (MPa)":315.0,
    "min(Rp0.2@400) (MPa)":null
  }
]


The import of data stored in JSON files into Pandas DataFrames is also possible and works as follows:

In [52]:
df = pd.read_json('steel-data.json').sort_index()
df.head()

,Type of Steel,EN,ASTM,UNS,JIS,c(Fe) (%),c(C) (%),c(N) (%),c(Cr) (%),c(Ni) (%),...,Others,Product,Rp0.2 (MPa),Rp1.0 (MPa),Rm (MPa),A (%),min(Rp0.2@100) (MPa),min(Rp0.2@200) (MPa),min(Rp0.2@300) (MPa),min(Rp0.2@400) (MPa)
0,Ferritic Steel,1.4600,None,None,None,87.930,0.020,0.0,11.2,0.85,...,Ti,H,435.0,460.0,545.0,25.0,375.0,NaN,NaN,NaN
1,Ferritic Steel,1.4512,409,None,SUS409,88.280,0.020,0.0,11.5,0.20,...,Ti,C,255.0,275.0,425.0,33.0,220.0,200.0,190.0,NaN
2,Ferritic Steel,1.4003,None,S40977,None,87.980,0.020,0.0,11.5,0.50,...,None,C,355.0,375.0,525.0,25.0,320.0,240.0,230.0,NaN
3,Ferritic Steel,1.4000,410S,S41008,SUS 403,87.470,0.030,0.0,12.5,0.00,...,None,P,270.0,320.0,490.0,30.0,230.0,220.0,210.0,190.0
4,Ferritic Steel,1.4589,None,S42035,None,84.055,0.045,0.0,14.0,1.65,...,Ti,C,470.0,510.0,600.0,20.0,420.0,NaN,NaN,NaN


Note, however, that `NaN`s in the original data are now `None`s.


## XML
XML stands for "eXtensible Markup Language" and it is indeed very extensible and flexible in the way data is represented. XML uses so-called tags to structure the data: 

    <tag>value</tag>
    
This makes it often hard to read by humans, especially if the data is nested. XML is an almost ancient way of representing data and not considered state-of-the-art nowadays.

It is also limited in the characters than can be used for defining a tag, for which reason, we need to adjust our column headers before exporting to XML:

In [53]:
df_xml = df.copy()
# Replace not allowed characters in column headers:
df_xml.columns = [col.replace(" ","").replace("(", "").replace(")", "").replace("%", "Pct").replace("@", "") for col in df_xml.columns]

# Export to XML:
df_xml.to_xml("steel-data.xml", index=False)

The resulting XML file looks as follows:

In [54]:
with open('steel-data.xml', 'r') as f:
    content = f.read()
print(content[:224], '\n\n[...]')

<?xml version='1.0' encoding='utf-8'?>
<data>
  <row>
    <TypeofSteel>Ferritic Steel</TypeofSteel>
    <EN>1.46</EN>
    <ASTM/>
    <UNS/>
    <JIS/>
    <cFePct>87.93</cFePct>
    <cCPct>0.02</cCPct>
    <cNPct>0.0</cNPct 

[...]


It can also be imported back to a Pandas DataFrame:

In [55]:
pd.read_xml("steel-data.xml").head()

,TypeofSteel,EN,ASTM,UNS,JIS,cFePct,cCPct,cNPct,cCrPct,cNiPct,...,Others,Product,Rp0.2MPa,Rp1.0MPa,RmMPa,APct,minRp0.2100MPa,minRp0.2200MPa,minRp0.2300MPa,minRp0.2400MPa
0,Ferritic Steel,1.4600,None,None,None,87.930,0.020,0.0,11.2,0.85,...,Ti,H,435.0,460.0,545.0,25.0,375.0,NaN,NaN,NaN
1,Ferritic Steel,1.4512,409,None,SUS409,88.280,0.020,0.0,11.5,0.20,...,Ti,C,255.0,275.0,425.0,33.0,220.0,200.0,190.0,NaN
2,Ferritic Steel,1.4003,None,S40977,None,87.980,0.020,0.0,11.5,0.50,...,None,C,355.0,375.0,525.0,25.0,320.0,240.0,230.0,NaN
3,Ferritic Steel,1.4000,410S,S41008,SUS 403,87.470,0.030,0.0,12.5,0.00,...,None,P,270.0,320.0,490.0,30.0,230.0,220.0,210.0,190.0
4,Ferritic Steel,1.4589,None,S42035,None,84.055,0.045,0.0,14.0,1.65,...,Ti,C,470.0,510.0,600.0,20.0,420.0,NaN,NaN,NaN


Note, however, that the column headers would now need to be reverted to the original ones for data consistency.
## Storing as YAML file
YAML is a simplified markup language for data serialization, which was originally based on XML and the data structures used in Perl, Python, and C as well as the mail format introduced in RFC 2822. YAML is a recursive acronym for „YAML Ain’t Markup Language“. 

To export a Pandas DataFrame to YAML, one must first convert the DataFrame to a dict, as follows:

In [56]:
yaml.dump(df.to_dict(orient="records"), open('steel-data.yaml', 'w'), 
          Dumper=yaml.Dumper, default_flow_style=False, sort_keys=False)

The resulting file is readable and very pythonic because its strictly structured by indents:

In [57]:
with open('steel-data.yaml', 'r') as f:
    content = f.read()
print(content[:224], '\n\n[...]')

- Type of Steel: Ferritic Steel
  EN: 1.46
  ASTM: null
  UNS: null
  JIS: null
  c(Fe) (%): 87.93
  c(C) (%): 0.02
  c(N) (%): 0.0
  c(Cr) (%): 11.2
  c(Ni) (%): 0.85
  c(Mo) (%): 0.0
  c(Al) (%): 0.0
  c(Mn) (%): 0.0
  c(C 

[...]


After re-loading the data from the YAML file, it is first represented as dictionary which needs subsequent conversion to a Pandas DataFrame:

In [58]:
with open('steel-data.yaml', 'r') as f:
    data = yaml.load(f, Loader=yaml.Loader)
df = pd.DataFrame(data)
df.head()

,Type of Steel,EN,ASTM,UNS,JIS,c(Fe) (%),c(C) (%),c(N) (%),c(Cr) (%),c(Ni) (%),...,Others,Product,Rp0.2 (MPa),Rp1.0 (MPa),Rm (MPa),A (%),min(Rp0.2@100) (MPa),min(Rp0.2@200) (MPa),min(Rp0.2@300) (MPa),min(Rp0.2@400) (MPa)
0,Ferritic Steel,1.4600,None,None,None,87.930,0.020,0.0,11.2,0.85,...,Ti,H,435.0,460.0,545.0,25.0,375.0,NaN,NaN,NaN
1,Ferritic Steel,1.4512,409,None,SUS409,88.280,0.020,0.0,11.5,0.20,...,Ti,C,255.0,275.0,425.0,33.0,220.0,200.0,190.0,NaN
2,Ferritic Steel,1.4003,None,S40977,None,87.980,0.020,0.0,11.5,0.50,...,None,C,355.0,375.0,525.0,25.0,320.0,240.0,230.0,NaN
3,Ferritic Steel,1.4000,410S,S41008,SUS 403,87.470,0.030,0.0,12.5,0.00,...,None,P,270.0,320.0,490.0,30.0,230.0,220.0,210.0,190.0
4,Ferritic Steel,1.4589,None,S42035,None,84.055,0.045,0.0,14.0,1.65,...,Ti,C,470.0,510.0,600.0,20.0,420.0,NaN,NaN,NaN


---

## Extra: Storing as SQL database

Sometimes (especially when dealing with large numbers of data sets), it is useful to store the data in a database instead of a file. Databases have the advantage the they can be asked ("queried") with various and also nested statements and that they are widely used and readible by other applications. Complex databases typically run on database servers, e.g. MySQL, Microsoft SQL, Oracle, etc. In this exercise we will use the non-server database format `SQLite`, which is extremely lightweight and the most widely deployed type of SQL database. 

Storing the complete DataFrame as SQL table is as simple as follows:

In [ ]:
with sqlite3.connect('steel-data.sql') as c:
    df.to_sql("steeldata", c, if_exists = 'replace', index = False)

And retrieving the data is as simple:

In [ ]:
with sqlite3.connect('steel-data.sql') as c:
    df = pd.read_sql('SELECT * FROM "steeldata"', c)
df.head()

### SQL database with mutliple tables

For performance and organizational reasons, databases typically consist of multiple tables grouped by contained information.

In the present case, we will group by 
* steel designation (names), 
* chemical composition, and 
* mechanical properties.

As probably noted earlier, the column "Type of Steel" contains only 7 different values, which could be managed in a separate table. This makes the structure more complex but generally decreases maintenance efforts (e.g. in case a certain value should be changed for all occurrences). 

For this purpose, we create a new DataFrame that encodes the steel types:

In [ ]:
df2 = pd.DataFrame([v for v in list(df["Type of Steel"].unique())], columns=["Type of Steel"])

Now we create a new column in `df` that contains the encoded steel types:

In [ ]:
df["TypesOfSteelId"] = df["Type of Steel"].replace({v:k for k, v in df2['Type of Steel'].to_dict().items()})

Next we define the tables to be written to the database:

In [ ]:
tables = {
    'Names': ['TypesOfSteelId', 'EN', 'ASTM', 'UNS', 'JIS'],
    'Concentrations': cols_conc,
    'Properties': cols_prop
}

And finally we write the different tables to the same SQL database as before:

In [ ]:
with sqlite3.connect('steel-data.sql') as c:
    for table_name, columns in tables.items():
        df[columns].to_sql(table_name, c, if_exists = 'replace', index_label="id")
    df2.to_sql("TypesOfSteel", c, if_exists = 'replace', index_label="id")

# Update `tables` for reloading the data from the SQL database:
tables.update({"TypesOfSteel": None})

### Loading from SQL database

Individual tables can be loaded from the database using a standard SQL `SELECT` statement:

In [2]:
with sqlite3.connect('steel-data.sql') as c:
    df = pd.read_sql('SELECT * FROM "Concentrations"', c, index_col="id")
df.head()

,c(Fe) (%),c(C) (%),c(N) (%),c(Cr) (%),c(Ni) (%),c(Mo) (%),c(Al) (%),c(Mn) (%),c(Cu) (%),c(Si) (%)
id,,,,,,,,,,
0,87.930,0.020,0.0,11.2,0.85,0.00,0.0,0.0,0.0,0.0
1,88.280,0.020,0.0,11.5,0.20,0.00,0.0,0.0,0.0,0.0
2,87.980,0.020,0.0,11.5,0.50,0.00,0.0,0.0,0.0,0.0
3,87.470,0.030,0.0,12.5,0.00,0.00,0.0,0.0,0.0,0.0
4,84.055,0.045,0.0,14.0,1.65,0.25,0.0,0.0,0.0,0.0


Since the data is distributed over various tables now, we need to join them accordingly:

In [ ]:
df = pd.DataFrame()

with sqlite3.connect('steel-data.sql') as c:

    for i, table_name in enumerate(tables):
        
        # Read the data set:
        new = pd.read_sql('SELECT * FROM "'+table_name+'"', c, index_col="id")

        if i==0:
            # Init:
            df = new
        elif len(new)==len(df):
            # Join on PK and PK:
            df = pd.merge(df, new, on='id')
        elif table_name in [col.replace("Id", "") for col in df.columns]:
            # Join on FK (df) and PK (new):
            df = pd.merge(df, new, left_on=table_name+'Id', right_on="id")
            df = df.drop(columns=[table_name+'Id'])

df.head()